# Setup

(Mostly taked from main demo. No need to read)

In [ ]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEVELOPMENT_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    %pip install circuitsvis
    
    # PySvelte is an unmaintained visualization library, use it as a backup if circuitsvis isn't working
    # # Install another version of node that makes PySvelte work way faster
    # !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    # %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

In [ ]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

Using renderer: colab


In [ ]:
import circuitsvis as cv
# Testing that the library works
cv.examples.hello("Connor")

In [ ]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from torchtyping import TensorType as TT
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [ ]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

In [ ]:
torch.set_grad_enabled(False)

In [ ]:
from transformer_lens import evals
import matplotlib.pyplot as plt
import collections
import plotly.graph_objects as go

In [ ]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# useful for sanity checks
model = HookedTransformer.from_pretrained("attn-only-2l", device=device)

Loaded pretrained model attn-only-2l into HookedTransformer


# Models with more than one layer have an abrubt improvement in in-context learning

In [ ]:
def in_context_learning_score(model, tokens):
    loss_vec = model(tokens, return_type='loss', loss_per_token=True)
    return (loss_vec[..., 500] - loss_vec[..., 50]).mean()

In [ ]:
# Small batch size to avoid cuda memory issues on colab
pile_batch_size = 4
pile_dataloader = evals.make_pile_data_loader(tokenizer=model.tokenizer, batch_size=pile_batch_size)

In [ ]:
checkpoint_indices = [10, 25, 35, 60, -1]
for model_name in ["attn-only-1l", "attn-only-2l", "attn-only-3l"]:
    tokens_trained_on = []
    in_context_learning_scores = []
    for index in checkpoint_indices:
        model_for_this_checkpoint = HookedTransformer.from_pretrained(model_name, checkpoint_index=index, device=device)

        tokens_seen_for_this_checkpoint = model_for_this_checkpoint.cfg.checkpoint_value
        tokens_trained_on.append(tokens_seen_for_this_checkpoint)

        in_context_learning_score_for_this_checkpoint = 0
        # Use subset of dataset for the sake of time
        num_batches = 4000 // pile_batch_size
        for i, x in enumerate(pile_dataloader):
            tokens = x['tokens'].to(device)
            in_context_learning_score_for_this_checkpoint += in_context_learning_score(model_for_this_checkpoint, tokens).item()
            if i == num_batches:
                break
        in_context_learning_score_for_this_checkpoint /= num_batches
        in_context_learning_scores.append(in_context_learning_score_for_this_checkpoint)
    fig = px.line(x=tokens_trained_on, y=in_context_learning_scores, title=model_name, labels={"x":"Elapsed Training Tokens", "y":"In-Context Learning Scores"})
    fig.update_layout(yaxis_range=[-1.0,1.0])
    fig.show()

Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer


Loaded pretrained model attn-only-2l into HookedTransformer
Loaded pretrained model attn-only-2l into HookedTransformer
Loaded pretrained model attn-only-2l into HookedTransformer
Loaded pretrained model attn-only-2l into HookedTransformer
Loaded pretrained model attn-only-2l into HookedTransformer


Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer


# Induction Heads form in phase change (Prefix Matching Score)

In [ ]:
batch_size = 10
seq_len = 50
random_tokens = torch.randint(1000, 10000, (batch_size, seq_len)).to(model.cfg.device)
repeated_tokens = einops.repeat(random_tokens, "batch seq_len -> batch (2 seq_len)")
repeated_tokens[:, 0] = model.tokenizer.bos_token_id

In [ ]:
# hook copied from transformer lens main demo 
def induction_score_hook(
    pattern: TT["batch", "head_index", "dest_pos", "source_pos"],
    hook: HookPoint,
):
    # We take the diagonal of attention paid from each destination position to source positions seq_len-1 tokens back
    # (This only has entries for tokens with index>=seq_len)
    induction_stripe = pattern.diagonal(dim1=-2, dim2=-1, offset=1-seq_len)
    # Get an average score per head
    induction_score = einops.reduce(induction_stripe, "batch head_index position -> head_index", "mean")
    # Store the result.
    induction_score_store[hook.layer(), :] = induction_score

# We make a boolean filter on activation names, that's true only on attention pattern names.
pattern_hook_names_filter = lambda name: name.endswith("pattern")

In [ ]:
checkpoint_indices = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, -1]
for model_name in ["attn-only-1l", "attn-only-2l", "attn-only-3l"]:
    tokens_trained_on = []
    induction_scores = collections.defaultdict(list)
    for index in checkpoint_indices:
        # Load the model from the relevant checkpoint by index
        model_for_this_checkpoint = HookedTransformer.from_pretrained(model_name, checkpoint_index=index, device=device)

        tokens_seen_for_this_checkpoint = model_for_this_checkpoint.cfg.checkpoint_value
        tokens_trained_on.append(tokens_seen_for_this_checkpoint)

        # induction_score_hook will store it's results here
        induction_score_store = torch.zeros((model_for_this_checkpoint.cfg.n_layers, model_for_this_checkpoint.cfg.n_heads), device=model_for_this_checkpoint.cfg.device)

        model_for_this_checkpoint.run_with_hooks(
            repeated_tokens, 
            return_type=None, # For efficiency, we don't need to calculate the logits
            fwd_hooks=[(
                pattern_hook_names_filter,
                induction_score_hook
            )]
        )

        for layer in range(model_for_this_checkpoint.cfg.n_layers):
            for head in range(model_for_this_checkpoint.cfg.n_heads):
                induction_scores[str(layer) + ',' + str(head)].append(induction_score_store[layer][head].item())
    fig = go.Figure(layout={'title': model_name})
    fig.update_xaxes(title="Elapsed Training Tokens")
    fig.update_yaxes(title="Prefix Matching Score")
    for layer_head, scores in induction_scores.items():
        fig.add_trace(go.Scatter(x=tokens_trained_on, y=scores, name=layer_head))
    fig.update_layout(yaxis_range=[0.0,1.0])
    fig.show()

Loaded pretrained model attn-only-1l into HookedTransformer


Downloading:   0%|          | 0.00/205M [00:00<?, ?B/s]

Loaded pretrained model attn-only-1l into HookedTransformer


Downloading:   0%|          | 0.00/205M [00:00<?, ?B/s]

Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer


...

Loaded pretrained model attn-only-3l into HookedTransformer


Downloading:   0%|          | 0.00/216M [00:00<?, ?B/s]

Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer


Downloading:   0%|          | 0.00/216M [00:00<?, ?B/s]

Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer


# Loss Curves Diverge during Phase Change

In [ ]:
checkpoint_indices = [10, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, -1]
for model_name in ["attn-only-1l", "attn-only-2l", "attn-only-3l"]:
    tokens_trained_on = []
    losses = []
    for index in checkpoint_indices:
        model_for_this_checkpoint = HookedTransformer.from_pretrained(model_name, checkpoint_index=index, device=device)

        tokens_seen_for_this_checkpoint = model_for_this_checkpoint.cfg.checkpoint_value
        tokens_trained_on.append(tokens_seen_for_this_checkpoint)

        loss_for_this_checkpoint = 0
        num_batches = 500
        for i, x in enumerate(pile_dataloader):
            tokens = x['tokens'].to(device)
            loss_for_this_checkpoint += model_for_this_checkpoint(tokens, return_type='loss').item()
            if i == num_batches:
                break
        loss_for_this_checkpoint /= num_batches
        losses.append(loss_for_this_checkpoint)
    fig = px.line(x=tokens_trained_on, y=losses, title=model_name, labels={"x":"Elapsed Training Tokens", "y":"Loss (nats / token)"})
    fig.update_layout(yaxis_range=[2.0,8.0])
    fig.show()

Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer


Downloading:   0%|          | 0.00/205M [00:00<?, ?B/s]

Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer


Downloading:   0%|          | 0.00/205M [00:00<?, ?B/s]

Loaded pretrained model attn-only-1l into HookedTransformer


...

Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer


Downloading:   0%|          | 0.00/216M [00:00<?, ?B/s]

Loaded pretrained model attn-only-3l into HookedTransformer


Downloading:   0%|          | 0.00/216M [00:00<?, ?B/s]

Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer


# Per Token Loss Principal component Analysis

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

In [ ]:
# collect some examples
examples = []
num_examples = 1000 // pile_batch_size
for i, x in enumerate(pile_dataloader):
    tokens = x['tokens'].to(device)
    examples.append(tokens)
    if i == num_examples - 1:
        break
examples[0].shape

torch.Size([4, 1024])

In [ ]:
indices = torch.randint(0, examples[0].shape[-1]-1, (len(examples) * pile_batch_size,))
indices.shape

torch.Size([1000])

In [ ]:
checkpoint_indices = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, -1]
for model_name in ["attn-only-1l", "attn-only-2l", "attn-only-3l"]:
    loss_data_matrix = torch.zeros((len(checkpoint_indices), len(examples) * pile_batch_size))
    for pos, index in enumerate(checkpoint_indices):
        model_for_this_checkpoint = HookedTransformer.from_pretrained(model_name, checkpoint_index=index, device=device)

        loss_vec_store = torch.zeros((len(examples) * pile_batch_size, examples[0].shape[-1]-1))
        for i, ex in enumerate(examples):
            loss_vec = model_for_this_checkpoint(ex, return_type="loss", loss_per_token=True)
            loss_vec_store[i*pile_batch_size:i*pile_batch_size + pile_batch_size] = loss_vec.cpu()
        loss_sampled = loss_vec_store[torch.arange(loss_vec_store.shape[0]), indices]
        # I needed to put this on cpu to avoid cuda memory errors...
        loss_data_matrix[pos] = loss_sampled.cpu()
    loss_data_scaled = StandardScaler().fit_transform(loss_data_matrix)
    pca_features = pca.fit_transform(loss_data_scaled)
    fig = px.scatter(x=pca_features[:, 0], y=pca_features[:, 1])
    fig.show()

Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer
Loaded pretrained model attn-only-1l into HookedTransformer


Loaded pretrained model attn-only-2l into HookedTransformer
Loaded pretrained model attn-only-2l into HookedTransformer
Loaded pretrained model attn-only-2l into HookedTransformer
Loaded pretrained model attn-only-2l into HookedTransformer
Loaded pretrained model attn-only-2l into HookedTransformer
Loaded pretrained model attn-only-2l into HookedTransformer
Loaded pretrained model attn-only-2l into HookedTransformer
Loaded pretrained model attn-only-2l into HookedTransformer
Loaded pretrained model attn-only-2l into HookedTransformer
Loaded pretrained model attn-only-2l into HookedTransformer
Loaded pretrained model attn-only-2l into HookedTransformer
Loaded pretrained model attn-only-2l into HookedTransformer
Loaded pretrained model attn-only-2l into HookedTransformer


Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer
Loaded pretrained model attn-only-3l into HookedTransformer
